In [314]:
def create_bag(elements):
    return {e:elements.count(e) for e in set(elements)}

In [315]:
with open("test_input.txt","r") as f:
    test_input = f.read().split("\n")

with open("input.txt", "r") as f:
    real_input = f.read().split("\n")

test_input

['32T3K 765', 'T55J5 684', 'KK677 28', 'KTJJT 220', 'QQQJA 483']

In [316]:
precidence = "AKQT98765432J"[::-1]
precidence = {c:precidence.find(c)+2 for c in precidence}
get_precidence = lambda c: precidence[c]

def get_tiebrake_score(cards):
    score = 0
    for c in cards:
        score *= 100
        score += precidence[c]
    return score

[(line, get_tiebrake_score(line)) for line in map(lambda x: x.split()[0], test_input)]

[('32T3K', 403110413),
 ('T55J5', 1106060206),
 ('KK677', 1313070808),
 ('KTJJT', 1311020211),
 ('QQQJA', 1212120214)]

In [317]:

def parse_line(line):
    cards, bid = line.split()
    return create_bag(cards), get_tiebrake_score(cards), cards, int(bid)

def parse_lines(lines):
    return [parse_line(line) for line in lines]

parse_lines(test_input)

[({'T': 1, 'K': 1, '2': 1, '3': 2}, 403110413, '32T3K', 765),
 ({'T': 1, '5': 3, 'J': 1}, 1106060206, 'T55J5', 684),
 ({'6': 1, 'K': 2, '7': 2}, 1313070808, 'KK677', 28),
 ({'T': 2, 'K': 1, 'J': 2}, 1311020211, 'KTJJT', 220),
 ({'J': 1, 'A': 1, 'Q': 3}, 1212120214, 'QQQJA', 483)]

In [318]:
from enum import IntEnum
HAND = IntEnum("Hand", ["FIVE", "FOUR", "FULL", "THREE", "DOUBLE_PAIR", "PAIR", "SINGLE"][::-1])

def get_type(bag):
    counts = list(bag.values())
    if 5 in counts:
        return HAND.FIVE
    elif 4 in counts:
        return HAND.FOUR
    elif 3 in counts and 2 in counts:
        return HAND.FULL
    elif 3 in counts:
        return HAND.THREE
    elif counts.count(2) == 2:
        return HAND.DOUBLE_PAIR
    elif 2 in counts:
        return HAND.PAIR
    else:
        return HAND.SINGLE
    
    return score

get_type(parse_lines(test_input)[4][0])

<Hand.THREE: 4>

In [319]:
def get_hand_score(hand_type, tiebrake_score):
    s1, s2 = str(hand_type+0), str(tiebrake_score)
    s2 = s2.zfill(10)
    s1 = s1.zfill(2)
    if s1 == "": print(hand_type, tiebrake_score)
    return int(s1 + s2)

get_hand_score(get_type(parse_lines(test_input)[0][0]), parse_lines(test_input)[0][1])
[(cards, get_hand_score(get_type(bag), tie)) for bag, tie, cards, bid in parse_lines(test_input)]

[('32T3K', 20403110413),
 ('T55J5', 41106060206),
 ('KK677', 31313070808),
 ('KTJJT', 31311020211),
 ('QQQJA', 41212120214)]

In [320]:
def order_lines(lines):
    parsed = parse_lines(lines)
    return sorted(parsed, key=lambda x: get_hand_score(get_type(x[0]), x[1]))

order_lines(test_input)

[({'T': 1, 'K': 1, '2': 1, '3': 2}, 403110413, '32T3K', 765),
 ({'T': 2, 'K': 1, 'J': 2}, 1311020211, 'KTJJT', 220),
 ({'6': 1, 'K': 2, '7': 2}, 1313070808, 'KK677', 28),
 ({'T': 1, '5': 3, 'J': 1}, 1106060206, 'T55J5', 684),
 ({'J': 1, 'A': 1, 'Q': 3}, 1212120214, 'QQQJA', 483)]

In [321]:
def total_winnings(lines):
    ordered = order_lines(lines)
    return sum((i+1)*bid for i, (_,_,_,bid) in enumerate(ordered))

total_winnings(test_input)

6440

In [322]:
total_winnings(real_input)


247991952

In [323]:
# not 249532660, too high

In [341]:
def new_type(bag):
    joker_count = bag["J"] if "J" in bag else 0 
    inflated_bag = {c:v for c,v in bag.items() if c !="J"}
    inflated_bag["J"] = 0
    inflated_bag[max([c for c in inflated_bag.keys()], key=lambda x: inflated_bag[x])] += joker_count
    return get_type(inflated_bag)

new_type(parse_line(test_input[0])[0])

<Hand.PAIR: 2>

In [349]:
def get_new_hand_score(bag, tiebrake_score):
    old_score = str(get_hand_score(get_type(bag), tiebrake_score)).zfill(12)
    new_score = str(new_type(bag)+0).zfill(2)
    score =  int(new_score + old_score[2:])
    return score

sorted([(cards, get_new_hand_score(bag,tie)) for bag, tie, cards, bid in parse_lines(test_input)], key=lambda x: x[1])

[('32T3K', 20403110413),
 ('KK677', 31313070808),
 ('T55J5', 61106060206),
 ('QQQJA', 61212120214),
 ('KTJJT', 61311020211)]

In [350]:
def new_order_lines(lines):
    parsed = parse_lines(lines)
    return sorted(parsed, key=lambda x: get_new_hand_score(x[0], x[1]))

new_order_lines(test_input)

[({'T': 1, 'K': 1, '2': 1, '3': 2}, 403110413, '32T3K', 765),
 ({'6': 1, 'K': 2, '7': 2}, 1313070808, 'KK677', 28),
 ({'T': 1, '5': 3, 'J': 1}, 1106060206, 'T55J5', 684),
 ({'J': 1, 'A': 1, 'Q': 3}, 1212120214, 'QQQJA', 483),
 ({'T': 2, 'K': 1, 'J': 2}, 1311020211, 'KTJJT', 220)]

In [351]:
def total_winnings_2(lines):
    ordered = new_order_lines(lines)
    return sum((i+1)*bid for i, (_,_,_,bid) in enumerate(ordered))

total_winnings_2(test_input)

5905

In [352]:
total_winnings_2(real_input)

248750699